In [1]:
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from multiprocessing import Process, Manager
import fcntl
import json as js
import time

In [2]:
with open('/home/tung/carlasim/data/extracted_data/config.json', 'r') as js_file:
    configs = js.load(js_file)

root_dir = configs['root']
extracted_dir = os.path.join(root_dir, configs['extracted_dir'])

transition_period = configs['transition_period']
out_of_view_accept_period = configs['out_of_view_period']
bb_threshold = configs['bb_threshold']
num_cams = configs['num_cams']

run_name = 'trans_{}_oov_{}'.format(transition_period, out_of_view_accept_period)
if bb_threshold:
    bb_threshold_x = configs['bb_threshold_x']
    bb_threshold_y = configs['bb_threshold_y']
    run_name += '_bb_{}_{}'.format(bb_threshold_x, bb_threshold_y)
run_dir_path = os.path.join(extracted_dir, run_name)

training_data_dir = os.path.join(run_dir_path, configs['training_data_dir'])
os.makedirs(training_data_dir, exist_ok = True)
final_simulation_data_dir = os.path.join(run_dir_path, configs['final_simulation_data_dir'])
os.makedirs(final_simulation_data_dir, exist_ok = True)

carla_raw_output_dir = os.path.join(root_dir, configs['carla_raw_output_dir'])

In [46]:
def delete_old_files():
    for d in [training_data_dir, final_simulation_data_dir]:
        for f in os.listdir(d):
            os.remove(os.path.join(d, f))

In [3]:

def extraction(job_idx : int, file_list):
    def get_cam_entry_list(frame_entries, num_cams):
        cam_entry = None
        for cam_num in range(num_cams):
            current_entry = frame_entries[cam_num]
            current_entry = current_entry.replace('(', '')
            current_entry = current_entry.replace(')', '')
            
            if (current_entry != '-1, -1'):
                # if cam_num == 9:
                #     print(current_entry)
                temp = current_entry.split(', ')
                if bb_threshold:
                    if ((float(temp[3]) > bb_threshold_x) or (float(temp[4]) > bb_threshold_y)):
                        frame, coor_x, coor_y = (int(x) for x in temp[0:3])
                        cam_entry = (cam_num, (frame, coor_x, coor_y))
                    else:
                        cam_entry = (-1, -1)   
                else: 
                    frame, coor_x, coor_y = (int(x) for x in temp[0:3])
                    cam_entry = (cam_num, (frame, coor_x, coor_y))
                break
            else:
                cam_entry = (-1, -1)
        return cam_entry
    #file_list = list(file_list)

    
    trajectory_list = {}
    for cam_num in range(num_cams):
        trajectory_list[cam_num] = []

    total_num_frames = 0
    for file_idx, file_name in enumerate(tqdm(file_list)):
        if '_' not in file_name:
            continue
        file_path = os.path.join(carla_raw_output_dir, file_name)
        track = pd.read_csv(file_path, header = 0, sep = ',')
        num_frames = len(track.index)
        current_cam = -1
        trajectory = []

        last_seen_frame_idx = 9999999999999
        first_seen_frame_idx = -9999999999999
        in_out_of_view_accept_period = False
        in_transition_period = False
        
        for frame_idx in range(num_frames):
            row = track.iloc[frame_idx, 1:]
            frame_entry = get_cam_entry_list(row, num_cams = num_cams)

            total_num_frames += 1

            # If the pedestrian is seen in one of the cameras
            if frame_entry != (-1, -1):
                record = frame_entry[1]
                if current_cam == frame_entry[0]:
                    pass
                else:
                    if current_cam == -1:
                        current_cam = frame_entry[0]
                        first_seen_frame_idx = frame_idx
                    else:
                        trajectory_list[current_cam].append(
                            np.array(
                                list([
                                    np.array(temp_trajectory, dtype = int),
                                    current_cam,
                                    frame_entry[0],
                                    frame_idx - last_seen_frame_idx
                                ]),
                                dtype = object
                            )
                        )
                        current_cam = frame_entry[0]
                trajectory.append(frame_entry[1])
                
                last_seen_frame_idx = frame_idx
                in_out_of_view_accept_period = False
                in_transition_period = False
            else:
                if (frame_idx - last_seen_frame_idx == 1):
                    in_out_of_view_accept_period = True
                    in_transition_period = True

                if (frame_idx - last_seen_frame_idx) > out_of_view_accept_period:
                    in_out_of_view_accept_period = False
                    if len(trajectory) > 0:
                        temp_trajectory = trajectory.copy()
                        trajectory = []
                    

                if (frame_idx - last_seen_frame_idx) > transition_period:
                    in_transition_period = False
                    in_out_of_view_accept_period = False
                    current_cam = -1
                    last_seen_frame_idx = 9999999999999
                    first_seen_frame_idx = -9999999999999
                    temp_trajectory = []

            # print('frame: {}, l_frame: {}, ccam: {}, oov: {}, trans: {}'.format(
            #     frame_idx, 
            #     last_seen_frame_idx,
            #     current_cam,
            #     in_out_of_view_accept_period,
            #     in_transition_period
            # ))
                

            
    # for cam_num in range(num_cams):
    #     if len(trajectory_list[cam_num]) > 0:
    #         while True:
    #             try:
    #                 temp = np.array(trajectory_list[cam_num])
    #                 cam_extracted_file_path = os.path.join(training_data_dir, str(cam_num)) + '.npy'
    #                 if os.path.exists(cam_extracted_file_path):
    #                     with open(cam_extracted_file_path, 'rb') as f:
    #                         fcntl.flock(f, fcntl.LOCK_EX)
    #                         old_data = np.load(cam_extracted_file_path, allow_pickle = True)
    #                         temp = np.concatenate((old_data, temp), axis = 0)
    #                         fcntl.flock(f, fcntl.LOCK_UN)
    #                         f.close()
    #                 with open(cam_extracted_file_path, 'wb') as f:
    #                     fcntl.flock(f, fcntl.LOCK_EX)
    #                     np.save(cam_extracted_file_path, temp)
    #                     fcntl.flock(f, fcntl.LOCK_UN)
    #                     f.close()
    #                 break
    #             except:
    #                 time.sleep(2.4)
    print('Finish job: {}, total number of frames: {}'.format(job_idx, total_num_frames))
                
# total_file_list = os.listdir(carla_raw_output_dir)
# total_num_files = len(total_file_list)
# extraction(total_file_list)

In [5]:
num_procs = 35
total_file_list = os.listdir(carla_raw_output_dir)
total_num_files = len(total_file_list)

#delete_old_files()


jobs = []
for job_idx in range(num_procs):
    num_files = int(total_num_files / num_procs)
    start = job_idx * num_files
    end = (job_idx + 1) * num_files
    if job_idx == (num_procs - 1):
        end = -1
    file_list = total_file_list[start:end]
    # extraction(file_list)
    #break
    p = Process(target = extraction, args = (job_idx, file_list))
    jobs.append(p)
    p.start()

Finish job: 32, total number of frames: 969132
Finish job: 23, total number of frames: 1017255
Finish job: 31, total number of frames: 993675
Finish job: 2, total number of frames: 978606
Finish job: 19, total number of frames: 978276
Finish job: 1, total number of frames: 989038
Finish job: 5, total number of frames: 1018406
Finish job: 22, total number of frames: 983667
Finish job: 21, total number of frames: 1017499
Finish job: 18, total number of frames: 1039981
Finish job: 25, total number of frames: 1011265
Finish job: 20, total number of frames: 1017846
Finish job: 16, total number of frames: 1036018
Finish job: 8, total number of frames: 985778
Finish job: 3, total number of frames: 994373
Finish job: 0, total number of frames: 1025032
Finish job: 4, total number of frames: 1016644
Finish job: 30, total number of frames: 1015425
Finish job: 10, total number of frames: 1012185
Finish job: 26, total number of frames: 1024268
Finish job: 11, total number of frames: 1013525
Finish 

In [59]:
for cam_num in range(num_cams):
    try:
        cam_extracted_file_path = os.path.join(training_data_dir, str(cam_num)) + '.npy'
        with open(cam_extracted_file_path, 'rb') as f:
            old_data = np.load(cam_extracted_file_path, allow_pickle = True)
            print(cam_num, len(old_data), old_data.shape[0])
            f.close()
    except:
        pass

0 700 700
1 3016 3016
2 60 60
3 877 877
4 2084 2084
5 379 379
6 1588 1588
7 269 269
8 2313 2313
9 1893 1893
10 2405 2405
11 1114 1114
12 2514 2514
13 3362 3362
14 1828 1828
15 1437 1437


FileNotFoundError: [Errno 2] No such file or directory: '/home/tung/carlasim/data/extracted_data/trans_1500_oov_30/training_data/16.npy'

In [ ]:













        in_transition_period = {}
        in_out_of_view_accept_period = {}
        last_frame_num = {}
        first_frame_num = {}
        trajectory = {}
        current_cam = -1
        for cam_num in range(num_cams):
            currently_tracked[cam_num] = False
            in_transition_period[cam_num] = False
            in_out_of_view_accept_period[cam_num] = False
            last_frame_num[cam_num] = 99999999999
            first_frame_num[cam_num] = -9999999999

for cam_num, entry in enumerate(frame_entry_list):
                if entry == (-1, -1):
                    if (frame_idx - last_frame_num[cam_num] == 1):
                        in_out_of_view_accept_period[cam_num] = True
                        in_transition_period[cam_num] = True

                    if (frame_idx - last_frame_num[cam_num]) > out_of_view_accept_period:
                        in_out_of_view_accept_period[cam_num] = False
                        currently_tracked[cam_num] = False

                    if (frame_idx - last_frame_num[cam_num]) > transition_period:
                        in_transition_period[cam_num] = False
                        in_out_of_view_accept_period[cam_num] = False
                        last_frame_num[cam_num] = 99999999999
                        first_frame_num[cam_num] = -9999999999

                    trajectory[cam_num] = []
                else:
                    if not currently_tracked[cam_num]:
                        current_cam = cam_num
                        first_frame_num[cam_num] = frame_idx
                        currently_tracked[cam_num] = True
                        in_out_of_view_accept_period[cam_num] = False
                        in_transition_period[cam_num] = False

                    trajectory[cam_num].append((
                        entry[0] - first_frame_num[cam_num], 
                        entry[1],
                        entry[2]
                    ))
                    last_frame_num[cam_num] = frame_idx

In [ ]:
for cam_num in trajectory_list.keys():
    print(cam_num, trajectory_list[cam_num])

In [6]:
for cam_num in trajectory_list.keys():
    print(len(trajectory_list[cam_num]))
    if len(trajectory_list[cam_num]) > 0:
        temp = np.array(trajectory_list[cam_num])
        cam_extracted_file_path = os.path.join(extracted_tracks_for_training_dir, str(cam_num)) + '.npy'
        with open(cam_extracted_file_path, 'w') as file:
            np.save(cam_extracted_file_path, temp)

            

896
596
0
221
651
0
213
359
122
606
359
274
377
148
61
414
573
463


In [ ]:
for file_name in os.listdir('/home/tung/carlasim/data/extracted_tracks_archived'):
    a = os.path.join('/home/tung/carlasim/data/extracted_tracks_archived', file_name)
    b = np.load(a, allow_pickle = True)
    print(b, '\n\n\n')